# 0. Kurulum ve Doğrulama
Bu bölüm gerekli kütüphanelerin ve veri setinin doğru yüklenip yüklenmediğini kontrol eder.

In [ ]:
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt

print("Kütüphaneler başarıyla içe aktarıldı.")

try:
    url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
    df = pd.read_csv(url)
    print(f"Veri başarıyla yüklendi. Boyut: {df.shape}")
except Exception as e:
    print(f"Veri yüklenirken hata oluştu: {e}")


# Titanic: Felaketten Makine Öğrenimi

Bu proje, Titanic yolcularının hayatta kalma durumlarını tahmin etmek için geliştirilmiştir.
Adımlar:
1. Veri Yükleme
2. Veri Ön İşleme
3. Keşifsel Veri Analizi
4. Modelleme (Rastgele Orman - Random Forest)
5. Değerlendirme

## A) Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

## B) Veri Yükleme

In [ ]:
# Veriyi GitHub üzerinden direkt yüklenir.
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# İlk 5 satırı göster
df.head()

In [ ]:
# Veri seti boyutu
df.shape

In [ ]:
# Veri seti bilgileri
df.info()

## C) Veri Ön İşleme

In [ ]:
# Eksik verilerin kontrolü
df.isnull().sum()

In [ ]:
# Eksik verileri görselleştirme
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')

### Eksik Verilerin Doldurulması ve Gereksiz Sütunların Atılması
- **Age (Yaş)**: Eksik değerler ortalama ile doldurulacak.
- **Cabin (Kabin)**: Çok fazla eksik olduğu için sütun atılacak.
- **Embarked (Biniş Limanı)**: Eksik değerler en çok tekrar eden (mode) ile doldurulacak.

In [ ]:
# Age eksikleri doldurulur.
df['Age'].fillna(df['Age'].mean(), inplace=True)

# Cabin sütunu atılır.
df.drop('Cabin', axis=1, inplace=True)

# Embarked eksikleri (2 adet) doldurulur.
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Tekrar kontrol
df.isnull().sum()

### Kategorik Verilerin Dönüştürülmesi

In [ ]:
# Cinsiyet (Sex) sütunu sayısala çevrilir. (male=0, female=1)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Embarked sütunu One-Hot Encoding yapılır (C, Q, S)
embarked_dummies = pd.get_dummies(df['Embarked'], prefix='Embarked', drop_first=True)
df = pd.concat([df, embarked_dummies], axis=1)

# Gereksiz sütunlar atılır. (Name, Ticket, PassengerId, Embarked-eskisi)
df.drop(['Name', 'Ticket', 'PassengerId', 'Embarked'], axis=1, inplace=True)

df.head()

## D) Keşifsel Veri Analizi

In [ ]:
# Hayatta kalma dağılımı
sns.countplot(x='Survived', data=df)
plt.title('Hayatta Kalma Dağılımı (0: Hayır, 1: Evet)')
plt.show()

In [ ]:
# Cinsiyete göre hayatta kalma
sns.countplot(x='Survived', hue='Sex', data=df)
plt.title('Cinsiyete Göre Hayatta Kalma (0: Erkek, 1: Kadın)')
plt.show()

In [ ]:
# Korelasyon Matrisi
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Korelasyon Matrisi')
plt.show()

## E) Modelleme (Rastgele Orman - Random Forest)

In [ ]:
X = df.drop('Survived', axis=1)
y = df['Survived']

# Eğitim ve Test verisi ayırma (%70 - %30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modeli oluşturma ve eğitme
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Modeli ve sütun bilgileri kaydedilir. (Uygulamalar için gerekli)
import joblib
joblib.dump(rf_model, 'titanic_model.pkl')
joblib.dump(X.columns, 'model_columns.pkl')
print('Model ve sütun bilgileri kaydedildi: titanic_model.pkl, model_columns.pkl')

## F) Değerlendirme

In [ ]:
y_pred = rf_model.predict(X_test)

print("Doğruluk Skoru (Accuracy):", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:\n", classification_report(y_test, y_pred))

In [ ]:
# Karışıklık Matrisi (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Karışıklık Matrisi')
plt.show()

## G) Tahmin Uygulaması (Metin Tabanlı)
Aşağıdaki hücre, modeli kullanarak terminal/çıktı ekranı üzerinden tahmin yapmanızı sağlayan kodları içerir. Çalıştırmak için fonksiyonu çağırabilirsiniz.

In [ ]:
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Sabitler
MODEL_DOSYASI = 'titanic_model.pkl'
SUTUN_DOSYASI = 'model_columns.pkl'

def modeli_egit():
    print("Model dosyası bulunamadı. Model şimdi eğitiliyor...")
    try:
        url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
        veri_cercevesi = pd.read_csv(url)

        # Ön İşleme
        veri_cercevesi['Age'].fillna(veri_cercevesi['Age'].mean(), inplace=True)
        veri_cercevesi.drop('Cabin', axis=1, inplace=True)
        veri_cercevesi['Embarked'].fillna(veri_cercevesi['Embarked'].mode()[0], inplace=True)
        veri_cercevesi['Sex'] = veri_cercevesi['Sex'].map({'male': 0, 'female': 1})

        liman_kukla_degiskenleri = pd.get_dummies(veri_cercevesi['Embarked'], prefix='Embarked', drop_first=True)
        veri_cercevesi = pd.concat([veri_cercevesi, liman_kukla_degiskenleri], axis=1)
        veri_cercevesi.drop(['Name', 'Ticket', 'PassengerId', 'Embarked'], axis=1, inplace=True)

        X = veri_cercevesi.drop('Survived', axis=1)
        y = veri_cercevesi['Survived']

        rf_modeli = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_modeli.fit(X, y)
        # Ayrım yapılmadan gerekli tüm bilgiler kullanılır.

        joblib.dump(rf_modeli, MODEL_DOSYASI)
        joblib.dump(X.columns, SUTUN_DOSYASI)
        print("Model başarıyla eğitildi ve kaydedildi.")
        return rf_modeli, X.columns
    except Exception as hata:
        print(f"Model eğitimi sırasında hata oluştu: {hata}")
        return None, None

def veri_al():
    print("\n--- Yeni Yolcu Bilgileri ---")

    # Yolcu Sınıfı (Pclass)
    while True:
        try:
            yolcu_sinifi = int(input("Yolcu Sınıfı (1, 2, 3): "))
            if yolcu_sinifi in [1, 2, 3]: break
            print("Lütfen 1, 2 veya 3 giriniz.")
        except: print("Lütfen sayı giriniz.")

    # Cinsiyet (Sex)
    while True:
        cinsiyet_giris = input("Cinsiyet (E/K): ").upper()
        if cinsiyet_giris in ['E', 'K', 'M', 'F']:
            # E/M -> 0 (Erkek), K/F -> 1 (Kadın)
            secilen_cinsiyet = 0 if cinsiyet_giris in ['E', 'M'] else 1
            break
        print("Lütfen 'E' (Erkek) veya 'K' (Kadın) giriniz.")

    # Yaş (Age)
    while True:
        try:
            yas = float(input("Yaş: "))
            if 0 < yas < 120: break
            print("Geçerli bir yaş giriniz.")
        except: print("Lütfen sayı giriniz.")

    # Kardeş/Eş Sayısı (SibSp)
    while True:
        try:
            kardes_es_sayisi = int(input("Kardeş/Eş Sayısı (SibSp): "))
            if kardes_es_sayisi >= 0: break
        except: print("Lütfen sayı giriniz.")

    # Ebeveyn/Çocuk Sayısı (Parch)
    while True:
        try:
            ebeveyn_cocuk_sayisi = int(input("Ebeveyn/Çocuk Sayısı (Parch): "))
            if ebeveyn_cocuk_sayisi >= 0: break
        except: print("Lütfen sayı giriniz.")

    # Bilet Ücreti (Fare)
    while True:
        try:
            ucret = float(input("Bilet Ücreti (Fare): "))
            if ucret >= 0: break
        except: print("Lütfen sayı giriniz.")

    # Biniş Limanı (Embarked)
    while True:
        binis_limani = input("Biniş Limanı (C = Cherbourg, Q = Queenstown, S = Southampton): ").upper()
        if binis_limani in ['C', 'Q', 'S']: break
        print("Lütfen C, Q veya S giriniz.")

    # İngilizce - Türkçe geçiş
    return {
        'Pclass': yolcu_sinifi,
        'Sex': secilen_cinsiyet,
        'Age': yas,
        'SibSp': kardes_es_sayisi,
        'Parch': ebeveyn_cocuk_sayisi,
        'Fare': ucret,
        'Embarked': binis_limani
    }

def # tahmin_yap()  # <--- Çalıştırmak için bu satırın başındaki yorum işareti kaldırılır:
    if not os.path.exists(MODEL_DOSYASI) or not os.path.exists(SUTUN_DOSYASI):
        model, model_sutunlari = modeli_egit()
        if model is None: return
    else:
        model = joblib.load(MODEL_DOSYASI)
        model_sutunlari = joblib.load(SUTUN_DOSYASI)

    while True:
        kullanici_verisi = veri_al()

        # DataFrame oluştur
        girdi_df = pd.DataFrame([kullanici_verisi])

        # Manuel Preprocessing (One-Hot Encoding)
        girdi_df['Embarked_Q'] = 1 if kullanici_verisi['Embarked'] == 'Q' else 0
        girdi_df['Embarked_S'] = 1 if kullanici_verisi['Embarked'] == 'S' else 0

        # Modelin beklediği sütun sırasına göre düzenle
        girdi_df = girdi_df.reindex(columns=model_sutunlari, fill_value=0)

        # Tahmin
        tahmin = model.predict(girdi_df)
        olasilik = model.predict_proba(girdi_df)[0]

        sonuc_metni = "HAYATTA KALDI" if tahmin[0] == 1 else "HAYATINI KAYBETTİ"
        guven_orani = olasilik[tahmin[0]] * 100

        print(f"\n**********************************")
        print(f"SONUÇ: {sonuc_metni}")
        print(f"Güven Oranı: %{guven_orani:.2f}")
        print(f"**********************************")

        devam = input("\nBaşka bir yolcu sorgulamak ister misiniz? (E/H): ").upper()
        if devam != 'E':
            print("Programdan çıkılıyor...")
            break

# if __name__ == "__main__":
    print("Titanic Hayatta Kalma Tahmin Aracı")
    # tahmin_yap()  # <--- Çalıştırmak için bu satırın başındaki yorum işareti kaldırılır.

> **Not:** Uygulamayı başlatmak için yukarıdaki hücreyi çalıştırdıktan sonra `tahmin_yap()` fonksiyonunu çağıran yeni bir hücre ekleyip çalıştırabilirsiniz.

## H) Tahmin Arayüzü (Görsel - GUI)
Aşağıdaki hücre, Tkinter kütüphanesini kullanarak görsel bir arayüz açar. Jupyter Notebook içinde çalıştırıldığında ayrı bir pencere açılacaktır.

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestClassifier

# Dosya adları ve sabitler
MODEL_DOSYASI = 'titanic_model.pkl'
SUTUN_DOSYASI = 'model_columns.pkl'

class TitanicTahminUygulamasi:
    def __init__(self, ana_pencere):
        self.ana_pencere = ana_pencere
        self.ana_pencere.title("Titanic Hayatta Kalma Tahmini")
        self.ana_pencere.geometry("400x550")

        # Stil ayarları
        style = ttk.Style()
        style.configure('TLabel', font=('Helvetica', 10))
        style.configure('TButton', font=('Helvetica', 10, 'bold'))

        # Başlık
        baslik_etiketi = tk.Label(ana_pencere, text="Titanic Tahmin Aracı", font=('Helvetica', 16, 'bold'))
        baslik_etiketi.pack(pady=10)

        # Form alanı
        self.form_cercevesi = ttk.Frame(ana_pencere, padding="20")
        self.form_cercevesi.pack(fill=tk.BOTH, expand=True)

        self.bilesenleri_olustur()

        # Model kontrolü
        self.model_yukle_veya_egit()

    def bilesenleri_olustur(self):
        # Yolcu Sınıfı (Pclass)
        ttk.Label(self.form_cercevesi, text="Yolcu Sınıfı:").grid(row=0, column=0, sticky=tk.W, pady=5)
        self.yolcu_sinifi_degiskeni = tk.StringVar()
        self.yolcu_sinifi_kombosu = ttk.Combobox(self.form_cercevesi, textvariable=self.yolcu_sinifi_degiskeni, state="readonly")
        self.yolcu_sinifi_kombosu['values'] = ('1. Sınıf', '2. Sınıf', '3. Sınıf')
        self.yolcu_sinifi_kombosu.current(2)
        self.yolcu_sinifi_kombosu.grid(row=0, column=1, sticky=tk.E, pady=5)

        # Cinsiyet (Sex)
        ttk.Label(self.form_cercevesi, text="Cinsiyet:").grid(row=1, column=0, sticky=tk.W, pady=5)
        self.cinsiyet_degiskeni = tk.StringVar()
        self.cinsiyet_kombosu = ttk.Combobox(self.form_cercevesi, textvariable=self.cinsiyet_degiskeni, state="readonly")
        self.cinsiyet_kombosu['values'] = ('Erkek', 'Kadın')
        self.cinsiyet_kombosu.current(0)
        self.cinsiyet_kombosu.grid(row=1, column=1, sticky=tk.E, pady=5)

        # Yaş (Age)
        ttk.Label(self.form_cercevesi, text="Yaş:").grid(row=2, column=0, sticky=tk.W, pady=5)
        self.yas_girisi = ttk.Entry(self.form_cercevesi)
        self.yas_girisi.insert(0, "30")
        self.yas_girisi.grid(row=2, column=1, sticky=tk.E, pady=5)

        # Kardeş/Eş Sayısı (SibSp)
        ttk.Label(self.form_cercevesi, text="Kardeş/Eş Sayısı:").grid(row=3, column=0, sticky=tk.W, pady=5)
        self.kardes_es_girisi = ttk.Entry(self.form_cercevesi)
        self.kardes_es_girisi.insert(0, "0")
        self.kardes_es_girisi.grid(row=3, column=1, sticky=tk.E, pady=5)

        # Ebeveyn/Çocuk Sayısı (Parch)
        ttk.Label(self.form_cercevesi, text="Ebeveyn/Çocuk Sayısı:").grid(row=4, column=0, sticky=tk.W, pady=5)
        self.ebeveyn_cocuk_girisi = ttk.Entry(self.form_cercevesi)
        self.ebeveyn_cocuk_girisi.insert(0, "0")
        self.ebeveyn_cocuk_girisi.grid(row=4, column=1, sticky=tk.E, pady=5)

        # Bilet Ücreti (Fare)
        ttk.Label(self.form_cercevesi, text="Bilet Ücreti:").grid(row=5, column=0, sticky=tk.W, pady=5)
        self.ucret_girisi = ttk.Entry(self.form_cercevesi)
        self.ucret_girisi.insert(0, "32.0")
        self.ucret_girisi.grid(row=5, column=1, sticky=tk.E, pady=5)

        # Biniş Limanı (Embarked)
        ttk.Label(self.form_cercevesi, text="Biniş Limanı:").grid(row=6, column=0, sticky=tk.W, pady=5)
        self.liman_degiskeni = tk.StringVar()
        self.liman_kombosu = ttk.Combobox(self.form_cercevesi, textvariable=self.liman_degiskeni, state="readonly")
        self.liman_kombosu['values'] = ('Cherbourg (C)', 'Queenstown (Q)', 'Southampton (S)')
        self.liman_kombosu.current(2)
        self.liman_kombosu.grid(row=6, column=1, sticky=tk.E, pady=5)

        # Tahmin Butonu
        self.tahmin_butonu = ttk.Button(self.ana_pencere, text="TAHMİN ET", command=self.tahmin_et)
        self.tahmin_butonu.pack(pady=20)

        # Sonuç Alanı
        self.sonuc_etiketi = tk.Label(self.ana_pencere, text="", font=('Helvetica', 12, 'bold'))
        self.sonuc_etiketi.pack(pady=10)

    def modeli_yukle_veya_egit_mantigi(self):
        """Model yoksa eğitir, varsa yükler."""
        if not os.path.exists(MODEL_DOSYASI) or not os.path.exists(SUTUN_DOSYASI):
            self.durum_guncelle("Model eğitimi başlıyor... Lütfen bekleyin.")
            self.ana_pencere.update()
            try:
                url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
                veri_cercevesi = pd.read_csv(url)

                # Ön İşleme
                veri_cercevesi['Age'].fillna(veri_cercevesi['Age'].mean(), inplace=True)
                veri_cercevesi.drop('Cabin', axis=1, inplace=True)
                veri_cercevesi['Embarked'].fillna(veri_cercevesi['Embarked'].mode()[0], inplace=True)
                veri_cercevesi['Sex'] = veri_cercevesi['Sex'].map({'male': 0, 'female': 1})

                liman_kukla_degiskenleri = pd.get_dummies(veri_cercevesi['Embarked'], prefix='Embarked', drop_first=True)
                veri_cercevesi = pd.concat([veri_cercevesi, liman_kukla_degiskenleri], axis=1)
                veri_cercevesi.drop(['Name', 'Ticket', 'PassengerId', 'Embarked'], axis=1, inplace=True)

                X = veri_cercevesi.drop('Survived', axis=1)
                y = veri_cercevesi['Survived']

                rf_modeli = RandomForestClassifier(n_estimators=100, random_state=42)
                rf_modeli.fit(X, y)

                joblib.dump(rf_modeli, MODEL_DOSYASI)
                joblib.dump(X.columns, SUTUN_DOSYASI)

                self.model = rf_modeli
                self.model_sutunlari = X.columns
                self.durum_guncelle("Model başarıyla eğitildi.")
            except Exception as hata:
                messagebox.showerror("Hata", f"Model eğitimi başarısız: {hata}")
                self.model = None
                self.model_sutunlari = None
        else:
            self.model = joblib.load(MODEL_DOSYASI)
            self.model_sutunlari = joblib.load(SUTUN_DOSYASI)

    def model_yukle_veya_egit(self):
        #İlgili fonk çağrılır.
        self.modeli_yukle_veya_egit_mantigi()

    def durum_guncelle(self, mesaj):
        self.sonuc_etiketi.config(text=mesaj, fg="blue")

    def veri_al_ve_hazirla(self):
        try:
            # Kullanıcı girdilerini al
            yolcu_sinifi_harita = {'1. Sınıf': 1, '2. Sınıf': 2, '3. Sınıf': 3}
            secilen_sinif = yolcu_sinifi_harita[self.yolcu_sinifi_degiskeni.get()]

            cinsiyet_degeri = 0 if self.cinsiyet_degiskeni.get() == 'Erkek' else 1

            yas = float(self.yas_girisi.get())
            kardes_es = int(self.kardes_es_girisi.get())
            ebeveyn_cocuk = int(self.ebeveyn_cocuk_girisi.get())
            ucret = float(self.ucret_girisi.get())

            liman_harita = {'Cherbourg (C)': 'C', 'Queenstown (Q)': 'Q', 'Southampton (S)': 'S'}
            secilen_liman = liman_harita[self.liman_degiskeni.get()]

            # Veri sözlüğü
            kullanici_verisi = {
                'Pclass': secilen_sinif,
                'Sex': cinsiyet_degeri,
                'Age': yas,
                'SibSp': kardes_es,
                'Parch': ebeveyn_cocuk,
                'Fare': ucret,
                'Embarked': secilen_liman
            }
            return kullanici_verisi
        except ValueError:
            messagebox.showwarning("Uyarı", "Lütfen tüm alanlara geçerli sayısal değerler giriniz.")
            return None

    def tahmin_et(self):
        if self.model is None:
            messagebox.showerror("Hata", "Model yüklenemedi!")
            return

        veri = self.veri_al_ve_hazirla()
        if veri is None:
            return

        # DataFrame oluştur
        girdi_df = pd.DataFrame([veri])

        # Manuel Preprocessing (One-Hot Encoding - Eğitimdeki ile uyumlu)
        girdi_df['Embarked_Q'] = 1 if veri['Embarked'] == 'Q' else 0
        girdi_df['Embarked_S'] = 1 if veri['Embarked'] == 'S' else 0

        # Modelin beklediği sütun sırasına göre düzenle
        # Eksik sütun varsa 0 ile doldur
        girdi_df = girdi_df.reindex(columns=self.model_sutunlari, fill_value=0)

        # Tahmin
        tahmin = self.model.predict(girdi_df)
        olasilik = self.model.predict_proba(girdi_df)[0]

        sonuc_metni = "HAYATTA KALDI" if tahmin[0] == 1 else "HAYATINI KAYBETTİ"
        guven_orani = olasilik[tahmin[0]] * 100

        renk = "green" if tahmin[0] == 1 else "red"

        self.sonuc_etiketi.config(
            text=f"Sonuç: {sonuc_metni}\nGüven Oranı: %{guven_orani:.2f}",
            fg=renk
        )

if __name__ == "__main__":
    kok = tk.Tk()
    uygulama = TitanicTahminUygulamasi(kok)
    kok.mainloop()
